# Multi-Agent Deep Research Assistant with Strands and AWS Bedrock

**Author:** Tutorial by [Manus AI](https://manus.im/app)

**Supervisor:** [Yan Xu](https://yanxuhappygela.github.io/yan-xu/)

**Framework:** Strands Agents SDK  

**Backend:** AWS Bedrock API  

**Difficulty level:** Itermediate to Advanced

---

## Overview

This notebook demonstrates how to build a **multi-agent deep research assistant** using the Strands agent framework with AWS Bedrock as the backend. The system takes a research topic from the user and generates a comprehensive research report through collaborative agent teamwork.

### What You'll Learn

- How to set up Strands Agents with AWS Bedrock
- How to create specialized agents with custom tools
- How to implement the Swarm pattern for multi-agent collaboration
- How to build a complete deep research system with 5 agents
- How agents autonomously hand off tasks to each other

### System Architecture

Our system uses **5 specialized agents** working collaboratively:

1. **Planning Agent**: Analyzes topics and creates research plans
2. **Research Agent**: Gathers information from multiple sources
3. **Analysis Agent**: Synthesizes data and extracts insights
4. **Writing Agent**: Creates well-structured research content
5. **Critic Agent**: Reviews and refines the final report

Each agent has **2-3 specialized tools** to accomplish its tasks.

### Why Strands
<a href="https://ibb.co/pjN0znz3"><img src="https://i.ibb.co/zVCnRNRS/Screenshot-2026-02-20-at-11-41-32-AM.png" alt="Screenshot-2026-02-20-at-11-41-32-AM" border="0"></a>

## 1. Installation and Setup

First, let's install the required packages.

In [ ]:
# Install required packages
!pip install -q strands-agents strands-agents-tools boto3 requests beautifulsoup4 ddgs

## 2. AWS Bedrock Configuration

Pre-requisite:
1. Setup AWS account (free credit $100): [register](https://signin.aws.amazon.com/signup?request_type=register)
2. Go to IAM (Identity and Access Management) to create a new user
3. Attach Policy "AmazonBedrockFullAccess" to the user
4. Go to created user and Click "Create access key"

Video tutorial: [Connect with Bedrock backend](https://youtu.be/-Wp3WdyzvXg?t=1078)

**Alternative option**
Setup a local ollama server running model locally:
[Build your first FREE AI Agent](https://colab.research.google.com/drive/12Djry5TI_NPxlzH0hnITT7lPfe-K0_JX?usp=sharing)

In [ ]:
import os
from google.colab import userdata

# Load AWS credentials from Colab secrets
os.environ["AWS_ACCESS_KEY_ID"] = userdata.get("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = userdata.get("AWS_SECRET_ACCESS_KEY")
os.environ["AWS_DEFAULT_REGION"] = userdata.get("AWS_DEFAULT_REGION")

## 3. Import Required Libraries

In [ ]:
from strands import Agent, tool
from strands.models import BedrockModel
from strands.multiagent import Swarm
import json
import logging
import boto3
from ddgs import DDGS
import requests
from bs4 import BeautifulSoup

# Configure logging
logging.getLogger("strands.multiagent").setLevel(logging.INFO)
logging.basicConfig(format='%(levelname)s | %(name)s | %(message)s', handlers=[logging.StreamHandler()])

## 4. LLM Invoke Function
Create a function to invoke Bedrock and parse the LLM output as json.

In [ ]:
import re
bedrock_client = boto3.client("bedrock-runtime")

def extract_json_from_text(text: str):
    """Extract JSON from LLM response that may contain markdown or extra text."""
    # Try to find JSON or list of JSON objects
    json_match = re.search(r'(\{.*\}|\[.*\])', text, re.DOTALL)
    if json_match:
        json_str = json_match.group(1)
        return json_str

    return text

def invoke_llm(prompt: str, model_id: str = "us.anthropic.claude-sonnet-4-20250514-v1:0", return_json: bool = True):
    """Invoke AWS Bedrock LLM with robust JSON extraction."""
    print(f'\n🤖 Invoking LLM for: {prompt[:80]}...')
    try:
        response = bedrock_client.invoke_model(
            modelId=model_id,
            contentType='application/json',
            accept='application/json',
            body=json.dumps({
                'messages': [{'role': 'user', 'content': prompt}],
                'max_tokens': 4096,
                'anthropic_version': 'bedrock-2023-05-31'
            })
        )
        result = json.loads(response.get('body').read())
        llm_output = result['content'][0]['text']
        print(f'✅ LLM response received.')

        if return_json:
            # Extract and parse JSON
            json_text = extract_json_from_text(llm_output)
            try:
                return json.loads(json_text)
            except json.JSONDecodeError as e:
                print(f'⚠️ JSON parsing failed, returning raw text. Error: {e}')
                return {"error": "JSON parsing failed", "raw_response": llm_output}
        else:
            return llm_output

    except Exception as e:
        print(f'❌ Error invoking LLM: {e}')
        return {"error": f"LLM invocation failed: {str(e)}"}

## 4. Define Tools for Each Agent

Each agent needs specialized tools to perform its tasks. Let's define them using the `@tool` decorator.

### File write and read tool

In [ ]:
# Universal save_output tool for all agents
@tool
def save_output(agent_name: str, content: str, stage: str = "output") -> dict:
    """
    Saves agent output to a markdown file for debugging and tracking.

    Args:
        agent_name: Name of the agent saving the output
        content: The content to save
        stage: Stage description (e.g., "plan", "research", "analysis", "draft", "final")

    Returns:
        dict with status and filename
    """
    import os
    from datetime import datetime

    # Create outputs directory if it doesn't exist
    os.makedirs('agent_outputs', exist_ok=True)

    # Generate filename with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"agent_outputs/{agent_name}_{stage}_{timestamp}.md"

    try:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(f"# {agent_name.replace('_', ' ').title()} - {stage.title()}\n\n")
            f.write(f"**Timestamp:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
            f.write("---\n\n")
            f.write(content)

        print(f"💾 {agent_name} saved output to: {filename}")
        return {
            "status": "success",
            "filename": filename,
            "size": len(content)
        }
    except Exception as e:
        print(f"❌ Failed to save output: {e}")
        return {
            "status": "error",
            "error": str(e)
        }


# Shared document tools for Writing and Critic agents
@tool
def append_to_report(section_title: str, content: str) -> dict:
    """
    Appends a section to the shared DEEP_RESEARCH_REPORT.md file.
    Used by Writing Agent to build the report section by section.

    Args:
        section_title: Title of the section being added
        content: The content to append

    Returns:
        dict with status and current file size
    """
    import os

    filename = "DEEP_RESEARCH_REPORT.md"

    try:
        # Check if file exists to determine if we need a header
        is_new_file = not os.path.exists(filename)

        with open(filename, 'a', encoding='utf-8') as f:
            if is_new_file:
                # Add document header for new file
                f.write("# Deep Research Report\n\n")
                f.write("*Generated by Multi-Agent Research Assistant*\n\n")
                f.write("---\n\n")

            # Append the section
            f.write(f"## {section_title}\n\n")
            f.write(content)
            f.write("\n\n")

        # Get current file size
        file_size = os.path.getsize(filename)

        print(f"📝 Appended '{section_title}' to {filename} (total size: {file_size} bytes)")

        return {
            "status": "success",
            "filename": filename,
            "section": section_title,
            "total_size": file_size
        }
    except Exception as e:
        print(f"❌ Failed to append to report: {e}")
        return {
            "status": "error",
            "error": str(e)
        }

@tool
def read_report() -> str:
    """
    Reads the complete DEEP_RESEARCH_REPORT.md file and returns the content as a string.
    Used by Critic Agent to review the full report.

    Returns:
        str: The complete report content, or error message if file doesn't exist
    """
    import os

    filename = "DEEP_RESEARCH_REPORT.md"

    try:
        if not os.path.exists(filename):
            error_msg = f"ERROR: Report file '{filename}' does not exist yet. Writing agent may not have completed."
            print(f"❌ {error_msg}")
            return error_msg

        with open(filename, 'r', encoding='utf-8') as f:
            content = f.read()

        file_size = os.path.getsize(filename)
        word_count = len(content.split())
        line_count = len(content.split('\n'))

        print(f"📖 Read {filename} ({file_size} bytes, {word_count} words, {line_count} lines)")

        # Return just the content as a string - this avoids Bedrock parsing issues
        return content

    except Exception as e:
        error_msg = f"ERROR: Failed to read report: {str(e)}"
        print(f"❌ {error_msg}")
        return error_msg

### Planning Agent Tools


*   topic_analyzer: Analyzes a research topic to identify key concepts, scope, and complexity
*   research_planner: Creates a structured research plan with subtopics and research questions.
*   outline_generator: Generates a preliminary outline for the research report.





In [ ]:
@tool
def topic_analyzer(topic: str) -> dict:
    """Analyzes a research topic to identify key concepts, scope, and complexity."""
    print(f'\n🔍 Analyzing topic: {topic}')
    prompt = f"""Analyze the research topic: "{topic}"
    Identify:
    1. Key concepts (3-5 main concepts)
    2. Scope (narrow/broad/comprehensive)
    3. Complexity level (beginner/intermediate/advanced)
    4. Research areas (3-5 primary areas to investigate)

    Return your analysis as a JSON object with this exact structure:
    {{
      "key_concepts": ["concept1", "concept2", "concept3"],
      "scope": "comprehensive",
      "complexity_level": "intermediate",
      "research_areas": ["area1", "area2", "area3", "area4"]
    }}

    Return ONLY the JSON object, no additional text."""

    return invoke_llm(prompt, return_json=True)

@tool
def research_planner(topic: str, key_concepts: list) -> dict:
    """Creates a structured research plan with subtopics and research questions."""
    print(f'\n📋 Creating research plan for: {topic}')
    prompt = f"""Create a structured research plan for: "{topic}"
    Key concepts: {key_concepts}

    Generate:
    1. Subtopics (3-5 specific subtopics to explore)
    2. Research questions (3-5 specific questions to answer)
    3. Methodology (brief description of research approach)

    Return your plan as a JSON object with this exact structure:
    {{
      "subtopics": ["subtopic1", "subtopic2", "subtopic3", "subtopic4"],
      "research_questions": ["question1", "question2", "question3", "question4", "question5"],
      "methodology": "description of research approach"
    }}

    Return ONLY the JSON object, no additional text."""

    return invoke_llm(prompt, return_json=True)

@tool
def outline_generator(research_plan: dict) -> str:
    """Generates a preliminary outline for the research report."""
    print('\n📝 Generating report outline')
    prompt = f"""Generate a detailed markdown outline for a research report based on this plan:
    {json.dumps(research_plan, indent=2)}

    Include these sections:
    1. Executive Summary
    2. Introduction
    3. Methodology
    4. Key Findings (with subsections for each subtopic from the plan)
    5. Analysis and Discussion
    6. Conclusions
    7. References

    Format the outline in markdown with proper heading levels (##, ###).
    Return ONLY the markdown outline, no additional commentary."""

    return invoke_llm(prompt, return_json=False)

In [ ]:
topic_analyzer("AI impact for workforce: future direction")


🔍 Analyzing topic: AI impact for workforce: future direction

🤖 Invoking LLM for: Analyze the research topic: "AI impact for workforce: future direction"
    Iden...
✅ LLM response received.


{'key_concepts': ['Artificial Intelligence',
  'Workforce Transformation',
  'Job Displacement and Creation',
  'Skills Evolution',
  'Future of Work'],
 'scope': 'comprehensive',
 'complexity_level': 'intermediate',
 'research_areas': ['AI Automation and Job Market Effects',
  'Reskilling and Workforce Development',
  'Human-AI Collaboration Models',
  'Economic and Social Policy Implications']}

### Research Agent Tools


*   web_search: Performs web search to find relevant information and sources.
*   content_extractor: Extracts and summarizes key information from a web source.



In [ ]:
@tool
def web_search(query: str, num_results: int = 5) -> list:
    """Performs web search to find relevant information and sources."""
    print(f'\n🔎 Searching web for: {query}')
    try:
        with DDGS() as ddgs:
            results = [r for r in ddgs.text(query, max_results=num_results)]
        print(f'✅ Found {len(results)} results.')
        return results
    except Exception as e:
        print(f'❌ Web search failed: {e}')
        return []


@tool
def content_extractor(url: str) -> dict:
    """Extracts and summarizes key information from a web source."""
    print(f'\n📄 Extracting content from: {url}')

    if url.endswith(".pdf"):
        return {
                    'error': str('pdf file'),
                    'url': url,
                    'summary': f'Could not parse pdf',
                    'key_points': ['Cannot parse pdf, skip']
                }

    # Try multiple times with increasing timeouts
    timeouts = [10, 20]

    for attempt, timeout in enumerate(timeouts, 1):
        try:
            print(f'  Attempt {attempt}/{len(timeouts)} (timeout: {timeout}s)...')

            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                'Accept-Language': 'en-US,en;q=0.5',
                'Accept-Encoding': 'gzip, deflate',
                'Connection': 'keep-alive',
            }

            response = requests.get(url, headers=headers, timeout=timeout, allow_redirects=True)
            response.raise_for_status()

            # Parse content
            soup = BeautifulSoup(response.content, 'html.parser')

            # Remove script and style elements
            for script in soup(["script", "style", "nav", "footer", "header"]):
                script.decompose()

            # Extract text from paragraphs first
            paragraphs = soup.find_all('p')
            text = ' '.join(p.get_text().strip() for p in paragraphs if p.get_text().strip())

            # Fallback to all text if no paragraphs
            if not text or len(text) < 100:
                text = soup.get_text(separator=' ', strip=True)

            # Clean up whitespace
            text = ' '.join(text.split())

            # Limit text length
            text = text[:10000]

            if not text or len(text) < 50:
                return {
                    'error': 'No meaningful content extracted',
                    'url': url,
                    'summary': 'Content extraction failed - page may be empty or require JavaScript',
                    'key_points': []
                }

            print(f'  ✅ Content extracted ({len(text)} chars)')

            # Use LLM to summarize
            prompt = f"""Summarize the following web content and extract the top 5 key points.
                      Content from {url}:
                      {text}
                      Return a JSON object with this exact structure:
                      {{
                        "summary": "brief summary of the content (2-3 sentences)",
                        "key_points": ["point1", "point2", "point3", "point4", "point5"]
                      }}
                      Return ONLY the JSON object, no additional text."""

            result = invoke_llm(prompt, return_json=True)

            # Add URL and metadata
            if isinstance(result, dict) and 'error' not in result:
                result['url'] = url
                result['content_length'] = len(text)
                return result
            else:
                return {'error': 'LLM summarization failed', 'url': url, 'raw_result': result}

        except requests.exceptions.Timeout:
            print(f'  ⏱️ Timeout after {timeout}s')
            if attempt == len(timeouts):
                # Last attempt failed
                return {
                    'error': f'Timeout after {len(timeouts)} attempts',
                    'url': url,
                    'summary': f'Could not extract content - website took longer than {timeouts[-1]} seconds to respond',
                    'key_points': ['Content extraction timed out', 'Website may be slow or unavailable']
                }
            # Try again with longer timeout
            continue

        except requests.exceptions.RequestException as e:
            print(f'  ❌ Request failed: {e}')
            return {
                'error': str(e),
                'url': url,
                'summary': f'Could not access website: {str(e)}',
                'key_points': ['Website access failed', 'May be blocked or unavailable']
            }

        except Exception as e:
            print(f'  ❌ Unexpected error: {e}')
            return {
                'error': str(e),
                'url': url,
                'summary': f'Content extraction failed: {str(e)}',
                'key_points': ['Extraction error occurred']
            }

In [ ]:
web_search("Top AI impact for workforce")


🔎 Searching web for: Top AI impact for workforce


✅ Found 5 results.


[{'title': 'How artificial intelligence impacts the US labor market | MIT Sloan',
  'href': 'https://mitsloan.mit.edu/ideas-made-to-matter/how-artificial-intelligence-impacts-us-labor-market',
  'body': 'October 9, 2025 - That’s because AI boosted firm productivity : Companies that used the technology grew faster, which helped sustain or even expand head count in high-exposure positions. The takeaway for employers rolling the technology out to their workforce: ...'},
 {'title': 'Evaluating the Impact of AI on the Labor Market: Current State of Affairs | The Budget Lab at Yale',
  'href': 'https://budgetlab.yale.edu/research/evaluating-impact-ai-labor-market-current-state-affairs',
  'body': 'As such, this metric attempts to capture how different the sum of occupations that make up the labor force is relative to another point in time. By measuring this over the time generative AI has been publicly available, we can test the claim that AI is substantially changing the workforce by any of

In [ ]:
content_extractor('https://www.deloitte.com/us/en/services/consulting/services/humans-x-machines.html')


📄 Extracting content from: https://www.deloitte.com/us/en/services/consulting/services/humans-x-machines.html
  Attempt 1/2 (timeout: 10s)...
  ✅ Content extracted (5867 chars)

🤖 Invoking LLM for: Summarize the following web content and extract the top 5 key points.
          ...
✅ LLM response received.


{'summary': "Deloitte's Humans x Machines service helps organizations combine human and AI capabilities to create exponential value through intentional work design. The approach focuses on leveraging the strengths of both humans and machines while building trust, measuring impact, and transforming work processes across industries.",
 'key_points': ["Workers are 5.2x more likely to champion AI tools when trust in employer's GenAI is high, emphasizing the critical role of trust in AI adoption",
  'Success requires intentional work design that combines human ingenuity with AI capabilities, moving beyond simple human-plus-machine to human-times-machine multiplication',
  'Deloitte offers comprehensive solutions including Workforce Impacts analysis, AI Integration Studio, and Human Capital AI Solution Suite to assess and implement human-machine collaboration',
  'The approach spans end-to-end transformation from strategy to execution, covering front-line operations, back office functions, a

### Analysis Agent Tools


*   insight_extractor: Extracts key insights and findings from the input data.
*   theme_organizer: Organizes findings into logical themes and categories.



In [ ]:
@tool
def insight_extractor(input_data: str) -> list:
    """Extracts key insights and findings from the input data."""
    print('\n💡 Extracting key insights')
    prompt = f"""From the input data, extract the top 3-5 most critical insights.

    Input data:
    {input_data[:10000]}

    For each insight provide:
    1. The insight itself
    2. Supporting evidence
    3. Significance level (High/Medium/Low)

    Return a JSON array with this exact structure:
    [
      {{
        "insight": "description of insight",
        "evidence": "supporting evidence",
        "significance": "High"
      }}
    ]

    Return ONLY the JSON array, no additional text."""

    return invoke_llm(prompt, return_json=True)

@tool
def theme_organizer(insights: list) -> dict:
    """Organizes findings into logical themes and categories."""
    print('\n🗂️ Organizing themes')
    prompt = f"""Organize these insights into 2-3 main themes for a research report.

    Insights:
    {json.dumps(insights, indent=2)}

    Return a JSON object where each key is a theme name and value contains:
    - description: brief description of the theme
    - insights: list of relevant insights from the input

    Structure:
    {{
      "Theme Name 1": {{
        "description": "theme description",
        "insights": ["insight1", "insight2"]
      }},
      "Theme Name 2": {{
        "description": "theme description",
        "insights": ["insight3", "insight4"]
      }}
    }}

    Return ONLY the JSON object, no additional text."""

    return invoke_llm(prompt, return_json=True)

### Writing Agent Tools


*   content_writer: Writes well-structured research content based on section and data.
*   citation_manager: Adds proper citations and manages references.
*   section_formatter: Formats sections according to research report standards.





In [ ]:
@tool
def content_writer(section: str, content_data: str) -> str:
    """Writes well-structured research content based on section and data."""
    print(f'\n✍️ Writing {section} section')
    prompt = f"""Write a comprehensive, well-structured section for a research report.

    Section: {section}
    Data to include:
    {content_data}

    Requirements:
    - Academic and objective tone
    - Well-organized paragraphs
    - Use markdown formatting
    - Include relevant details from the data

    Return ONLY the written content in markdown format, no additional commentary."""

    return invoke_llm(prompt, return_json=False)


@tool
def citation_manager(content: str, sources: list) -> str:
    """Adds proper citations and manages references."""
    print('\n📚 Adding citations...')
    cited_content = content
    reference_list = '\n\n## References\n\n'

    if not sources:
        return content

    for i, source in enumerate(sources, 1):
        if isinstance(source, dict) and 'url' in source:
            cited_content += f' [{i}]'
            title = source.get('title', 'Source')
            url = source.get('url', '')
            reference_list += f"{i}. {title} - <{url}>\n"

    return cited_content + reference_list


@tool
def section_formatter(content: str, section_type: str) -> str:
    """Formats sections according to research report standards."""
    print(f'\n📐 Formatting {section_type} section...')
    return f'\n---\n{content}\n---\n'

### Critic Agent Tools


*   quality_checker: Evaluates report quality, completeness, and coherence.
*   improvement_suggester: Suggests specific improvements and refinements.



In [ ]:
@tool
def quality_checker(report: str) -> dict:
    """Evaluates report quality, completeness, and coherence."""
    print('\n🔍 Checking report quality')
    prompt = f"""Evaluate this research report for quality, completeness, and coherence.

    Report:
    {report[:10000]}

    Provide:
    1. Quality score (0.0 to 1.0)
    2. Completeness score (0.0 to 1.0)
    3. Coherence score (0.0 to 1.0)
    4. Strengths (list of 2-4 strengths)
    5. Weaknesses (list of 2-4 weaknesses)

    Return a JSON object with this exact structure:
    {{
      "quality_score": 0.85,
      "completeness_score": 0.90,
      "coherence_score": 0.88,
      "strengths": ["strength1", "strength2"],
      "weaknesses": ["weakness1", "weakness2"]
    }}

    Return ONLY the JSON object, no additional text."""

    return invoke_llm(prompt, return_json=True)


@tool
def improvement_suggester(report: str, quality_assessment: dict) -> list:
    """Suggests specific improvements and refinements."""
    print('\n💡 Generating improvement suggestions')
    prompt = f"""Based on this quality assessment, provide specific, actionable suggestions to improve the report.

    Quality Assessment:
    {json.dumps(quality_assessment, indent=2)}

    For each suggestion provide:
    1. Priority (High/Medium/Low)
    2. Category (Content/Structure/Clarity)
    3. The suggestion itself

    Return a JSON array with this exact structure:
    [
      {{
        "priority": "High",
        "category": "Content",
        "suggestion": "specific actionable suggestion"
      }}
    ]

    Return ONLY the JSON array, no additional text."""

    return invoke_llm(prompt, return_json=True)

## 5. Create the Specialized Agents

Now we'll create our five specialized agents, each with their own system prompt and tools.

In [ ]:
# Configure the Bedrock model
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",
    region_name="us-east-1",
    temperature=0.7
)

# Planning Agent
planning_agent = Agent(
    name="planning_agent",
    system_prompt="""
      You are a research planning specialist. Your role is to:
      1. Analyze research topics to identify key concepts and scope
      2. Create structured research plans with subtopics and questions
      3. Generate preliminary outlines for research reports

      After completing your planning work:
      1. Use save_output to save your research plan and outline
      2. Hand off to the research_agent to begin information gathering
      Use your tools to thoroughly analyze and plan before handing off.
      """,
    tools=[topic_analyzer, research_planner, outline_generator, save_output],
    model=bedrock_model
)

# Research Agent
research_agent = Agent(
    name="research_agent",
    system_prompt="""
      You are a research specialist focused on gathering comprehensive information. Your role is to:
      1. Search for relevant information using web_search
      2. Extract key content from sources using content_extractor

      Gather information on all subtopics from the research plan. Once you have sufficient
      high-quality information:
      1. Use save_output to save all gathered research data and sources
      2. Hand off to the analysis_agent for synthesis
      """,
    tools=[web_search, content_extractor, save_output],
    model=bedrock_model
)

# Analysis Agent
analysis_agent = Agent(
    name="analysis_agent",
    system_prompt="""
      You are an analytical specialist who synthesizes research findings. Your role is to:
      1. Extract key insights using insight_extractor
      2. Organize findings into logical themes using theme_organizer

      Thoroughly analyze all gathered information and organize it logically. When analysis is
      complete:
      1. Use save_output to save your analysis, insights, and organized themes
      2. Hand off to the writing_agent to create the report
      """,
    tools=[insight_extractor, theme_organizer, save_output],
    model=bedrock_model
)

# Writing Agent
writing_agent = Agent(
    name="writing_agent",
    system_prompt="""
      You are a professional research writer who creates well-structured reports. Your role is
      Create a complete research report based on the analysis. Write each section carefully with
      proper structure and citations:
      1. Write clear, comprehensive content using content_writer for each section
      2. Add proper citations using citation_manager
      3. Format sections professionally using section_formatter
      4. Append each completed section to DEEP_RESEARCH_REPORT.md using append_to_report

      IMPORTANT:
      - Build the report section by section, appending each one to DEEP_RESEARCH_REPORT.md
      - Hand off to the critic_agent for review
      - The critic will read the complete report from DEEP_RESEARCH_REPORT.md

      Do NOT output the full report in your response - it's being built in the file.
      Just confirm completion and hand off to critic_agent.
      """,
    tools=[content_writer, citation_manager, section_formatter, append_to_report],
    model=bedrock_model
)

# Critic Agent
critic_agent = Agent(
    name="critic_agent",
    system_prompt="""
      You are a critical reviewer and quality assurance specialist. Your role is to:
      1. Read the complete report from DEEP_RESEARCH_REPORT.md using read_report
      2. Evaluate report quality using quality_checker
      3. Suggest improvements using improvement_suggester
      4. If there are major issues (quality score < 0.8):
        - Hand back to writing_agent for content issues
        - Hand back to research_agent for missing information
      5. If the report meets quality standards (score >= 0.8):
        - Use save_output to save your assessment

      Provide your final assessment in this format:
      ---
      ## Quality Assessment

      **Report File:** DEEP_RESEARCH_REPORT.md

      **Quality Score:** [score]
      **Completeness Score:** [score]
      **Coherence Score:** [score]

      ### Strengths
      - [strength 1]
      - [strength 2]
      - [strength 3]

      ### Areas for Improvement
      - [suggestion 1]
      - [suggestion 2]

      ### Verdict
      [APPROVED / NEEDS REVISION]

      ---

      IMPORTANT:
      - Read the report from DEEP_RESEARCH_REPORT.md, don't expect it in the handoff message
      - The complete report is in the file, not in your context
      - Your assessment should reference the file content
      - Do not hand off after approval - this is the final step
      """,
          tools=[read_report, quality_checker, improvement_suggester, save_output],
          model=bedrock_model
)

print("✅ All agents created successfully!")

✅ All agents created successfully!


## 6. Create and Configure the Swarm

Now we'll create the Swarm that orchestrates our multi-agent system.

In [ ]:
# Create the swarm with all agents
research_swarm = Swarm(
    [
        planning_agent,
        research_agent,
        analysis_agent,
        writing_agent,
        critic_agent
    ],
    entry_point=planning_agent,  # Start with planning
    max_handoffs=25,  # Allow sufficient handoffs for collaboration
    max_iterations=30,  # Total iterations across all agents
    execution_timeout=1800.0,  # 30 minutes for complex research
    node_timeout=600.0,  # 10 minutes per agent
    repetitive_handoff_detection_window=8,
    repetitive_handoff_min_unique_agents=3
)

print("✅ Swarm created successfully!")
print(f"\nSwarm Configuration:")
print(f"  - Entry point: {planning_agent.name}")
print(f"  - Number of agents: 5")
print(f"  - Max handoffs: 25")
print(f"  - Max iterations: 30")

✅ Swarm created successfully!

Swarm Configuration:
  - Entry point: planning_agent
  - Number of agents: 5
  - Max handoffs: 25
  - Max iterations: 30


## 7. Execute the Research Swarm

Let's run our multi-agent system on a research topic!

In [ ]:
# Define the research topic
research_topic = "The impact of artificial intelligence on workforce: future directions"

print(f"\n{'='*80}")
print(f"STARTING MULTI-AGENT DEEP RESEARCH")
print(f"{'='*80}")
print(f"\nTopic: {research_topic}")
print(f"\nThe agents will now collaborate to produce a comprehensive research report...\n")
print(f"{'='*80}\n")

# Execute the swarm
result = research_swarm(research_topic)

print(f"\n{'='*80}")
print(f"RESEARCH COMPLETED")
print(f"{'='*80}")


STARTING MULTI-AGENT DEEP RESEARCH

Topic: The impact of artificial intelligence on workforce: future directions

The agents will now collaborate to produce a comprehensive research report...


I'll help you create a comprehensive research plan for "The impact of artificial intelligence on workforce: future directions." Let me start by analyzing this topic to identify key concepts and scope.
Tool #1: topic_analyzer

🔍 Analyzing topic: The impact of artificial intelligence on workforce: future directions

🤖 Invoking LLM for: Analyze the research topic: "The impact of artificial intelligence on workforce:...
✅ LLM response received.
Now I'll create a structured research plan based on these key concepts:
Tool #2: research_planner

📋 Creating research plan for: The impact of artificial intelligence on workforce: future directions

🤖 Invoking LLM for: Create a structured research plan for: "The impact of artificial intelligence on...
✅ LLM response received.
Now let me generate a prelimina

## 8. View Results and Analysis

Let's examine the results of our multi-agent collaboration.

In [ ]:
# Display final status
print(f"\n📊 EXECUTION SUMMARY")
print(f"{'='*80}")
print(f"Status: {result.status}")
print(f"Total agents involved: {len(result.node_history)}")
print(f"\nAgent collaboration sequence:")
for i, node in enumerate(result.node_history, 1):
    print(f"  {i}. {node.node_id}")

print(f"\n{'='*80}")


📊 EXECUTION SUMMARY
Status: Status.COMPLETED
Total agents involved: 5

Agent collaboration sequence:
  1. planning_agent
  2. research_agent
  3. analysis_agent
  4. writing_agent
  5. critic_agent



## 9. Conclusion

You've successfully built a multi-agent deep research assistant using:

- **Strands Agents SDK** for agent framework
- **AWS Bedrock** for LLM backend (Claude 4 Sonnet)
- **Swarm Pattern** for autonomous multi-agent collaboration
- **5 Specialized Agents** with 15 custom tools

### Key Takeaways

1. **Agent Specialization**: Each agent has a clear role and specialized tools
2. **Autonomous Collaboration**: Agents decide when to hand off tasks
3. **Shared Context**: All agents have access to the full conversation history
4. **Tool-Based Coordination**: The `handoff_to_agent` tool enables seamless transitions
5. **Scalable Architecture**: Easy to add more agents or tools as needed
6. **Use **file system** efficiently to manage tokens efficiently

### Next Steps
- Enhance instruction for Writing agent
- Improve interactions with file system to manage context efficiently and optimize token usage
- Add more specialized agents for your domain
- Implement evaluation metrics
- Explore other Strands patterns (Graph, Workflow)

### Resources

- [Strands Agents Documentation](https://strandsagents.com/)
- [AWS Bedrock Documentation](https://aws.amazon.com/bedrock/)
- [Strands GitHub Repository](https://github.com/strands-agents/sdk-python)

---

**Happy Building! 🚀**